In [11]:
from dotenv import load_dotenv
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
import shutil
import os

# suggested structure:
# TUM-ai_Makeathon
# ├── data
# │   ├── bronze_layer
# │   │   ├── EuroSAT_MS
# │   ├── silver_layer
# │   ├── gold_layer
local_directory = "data/temp_layer" # TODO: change this on your purpose to the directory you wanna pull form the blob storage

dotenv_path = '.env'
load_dotenv(dotenv_path)
connection_string = os.getenv('BLOB_STORAGE_CONNECTION_STRING')

container_name = "data-container"
blob_folder_path = local_directory
blob_folder_path = blob_folder_path.replace("data/", "")  # remove the data/ prefix

assert blob_folder_path != "", "blob_folder_path cannot be empty"
assert local_directory != "", "local_directory cannot be empty"

if os.path.exists(local_directory):
    shutil.rmtree(local_directory)
os.makedirs(local_directory)

blob_service_client = BlobServiceClient.from_connection_string(connection_string)
container_client = blob_service_client.get_container_client(container=container_name)


blob_list = container_client.list_blobs()

for blob in blob_list:
    if not blob.name.startswith(blob_folder_path):
        continue
    # print full paths of the blobs
    blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob.name)

    # create folders and subfolders and file
    os.makedirs(os.path.dirname(f"data/{blob.name}"), exist_ok=True)
    with open(f"data/{blob.name}", "wb") as file:
        download_stream = blob_client.download_blob()
        file.write(download_stream.readall())